In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dense, Input, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import os
import zipfile

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /content/cats_and_dogs_filtered.zip

--2022-04-19 13:03:07--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.128, 209.85.145.128, 209.85.147.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/content/cats_and_dogs_filtered.zip’

/content/cats_and_d 100%[===================>]  65.43M   215MB/s    in 0.3s    

2022-04-19 13:03:07 (215 MB/s) - ‘/content/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
local_zip = '/content/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
base_dir = '/content/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
train_cat_fnames = os.listdir(train_cats_dir)
print(train_cat_fnames[:10])

train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
print(train_dog_fnames[:10])

['cat.456.jpg', 'cat.710.jpg', 'cat.948.jpg', 'cat.325.jpg', 'cat.884.jpg', 'cat.569.jpg', 'cat.270.jpg', 'cat.536.jpg', 'cat.105.jpg', 'cat.249.jpg']
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg', 'dog.106.jpg']


In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500


In [ ]:
train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rescale=1./255,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=20,
                                                    class_mode='binary')
test_generator = test_datagen.flow_from_directory(validation_dir,
                                                  target_size=(224,224),
                                                  batch_size=20,
                                                  class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
#VGG16 Model
def VGG16_Model(shape, classes):
  #Block 1
  input_shape = Input(shape=shape)
  x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(input_shape)
  x = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = MaxPooling2D(2, strides=2, padding='same')(x)
  x = BatchNormalization()(x)

  #Block 2
  x = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = MaxPooling2D(2, strides=2, padding='same')(x)
  x = BatchNormalization()(x)

  #Block 3
  x = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = MaxPooling2D(2, strides=2, padding='same')(x)
  x = BatchNormalization()(x)

  #Block 4
  x = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = MaxPooling2D(2, strides=2, padding='same')(x)
  x = BatchNormalization()(x)

  #Block 5
  x = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu')(x)
  x = MaxPooling2D(2, strides=2, padding='same')(x)
  x = BatchNormalization()(x)
  x = GlobalAveragePooling2D()(x)

  x = Flatten()(x)
  x = Dense(512, activation='relu')(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.5)(x)
  if classes==2:
    activation_fn='sigmoid'
  else:
    activation_fn='softmax'
  output = Dense(1, activation=activation_fn)(x)

  model = Model(inputs=input_shape, outputs=output)

  return model 

In [ ]:
model = VGG16_Model(shape=(224,224,3), classes=2)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856 

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=15,
                              validation_data=test_generator,
                              validation_steps=50,
                              verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/15
100/100 [==============================] - 88s 883ms/step - loss: 0.7457 - accuracy: 0.5285 - val_loss: 0.7308 - val_accuracy: 0.5310
Epoch 2/15
100/100 [==============================] - 82s 815ms/step - loss: 0.7114 - accuracy: 0.5310 - val_loss: 0.8997 - val_accuracy: 0.5040
Epoch 3/15
100/100 [==============================] - 81s 814ms/step - loss: 0.7011 - accuracy: 0.5240 - val_loss: 0.7419 - val_accuracy: 0.5020
Epoch 4/15
100/100 [==============================] - 80s 801ms/step - loss: 0.6919 - accuracy: 0.5560 - val_loss: 0.6915 - val_accuracy: 0.5330
Epoch 5/15
100/100 [==============================] - 80s 802ms/step - loss: 0.6886 - accuracy: 0.5515 - val_loss: 0.7606 - val_accuracy: 0.5530
Epoch 6/15
100/100 [==============================] - 81s 809ms/step - loss: 0.6866 - accuracy: 0.5595 - val_loss: 0.6930 - val_accuracy: 0.4950
Epoch 7/15
100/100 [==============================] - 81s 809ms/step - loss: 0.6920 - accuracy: 0.5660 - val_loss: 0.9501 - val_ac

In [ ]:
model.evaluate_generator(test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.6844071745872498, 0.5329999923706055]

In [ ]:
model.save_weights('VGG16_weights.')